In [47]:
import copy
import numpy
from numpy import exp, square
na = numpy.newaxis
imag = numpy.imag
import scipy
import scipy.special
from scipy.special import erfc, exp1
import cryspy

f_name = "tof.rcif"
rhochi_obj = cryspy.file_to_globaln(f_name)

In [2]:
def calc_y_z_u_v(alpha, beta, sigma, delta_2d):
    """Calculate y, z, u, v

    y = (alpha * sigma**2 + delta)/(sigma * 2**0.5)
    z = (beta * sigma**2 - delta)/(sigma * 2**0.5)
    u = 0.5 * alpha * (alpha*sigma**2 + 2 delta)
    v = 0.5 * beta * (beta*sigma**2 - 2 delta)

    """
    sigma_sq = square(sigma)
    y = (alpha * sigma/(2.**0.5))[:, na] + delta_2d/(sigma*2.**0.5)[:, na]
    z = (beta * sigma/(2.**0.5))[:, na] - delta_2d/(sigma*2.**0.5)[:, na]
    u = (0.5*square(alpha)*sigma_sq)[:, na] + delta_2d*alpha[:, na]
    v = (0.5*square(beta)*sigma_sq)[:, na] - delta_2d*beta[:, na]
    return y, z, u, v

In [3]:
def calc_hpv_eta(h_g, h_l):
    """pseudo-Voight function
    
    calculate h_pV and eta based on Gauss and Lorentz Size
    """
    h_g_2, h_l_2 = h_g**2, h_l**2
    h_g_3, h_l_3 = h_g_2*h_g, h_l_2*h_l
    h_g_4, h_l_4 = h_g_3*h_g, h_l_3*h_l
    h_g_5, h_l_5 = h_g_4*h_g, h_l_4*h_l
    c_2, c_3, c_4, c_5 = 2.69269, 2.42843, 4.47163, 0.07842
    h_pv = (h_g_5 + c_2*h_g_4*h_l + c_3*h_g_3*h_l_2 + 
            c_4*h_g_2*h_l_3 + c_5*h_g*h_l_4 + h_l_5)**0.2
    hh = h_l*1./h_pv
    eta = 1.36603*hh - 0.47719*hh**2 + 0.11116*hh**3
    return h_pv, eta

In [11]:
def calc_sigma_gamma(
        d, sigma0, sigma1, sigma2, gamma0, gamma1, gamma2,
        size_g: float = 0., size_l: float = 0., strain_g: float = 0.,
        strain_l: float = 0.):
    """Calculate H_G (sigma) and H_L (gamma)
    
        H_G**2 = (sigma2+size_g)*d**4 + (sigma1+strain_g)*d**2 + sigma0
        H_L = (gamma2+size_l)*d**2 + (sigma1+strain_l)*d + sigma0
    """
    d_sq = numpy.square(d)
    d_sq_sq = numpy.square(d_sq)

    h_g_sq = (sigma2+size_g) * d_sq_sq + (sigma1+strain_l) * d_sq + sigma0
    h_l = (gamma2+size_l) * d_sq + (gamma1+strain_l) * d + gamma0
    h_g = numpy.sqrt(h_g_sq)
    return h_g, h_l

In [16]:

def calc_hpv_eta(h_g, h_l):
    """pseudo-Voight function
    
    calculate h_pV and eta based on Gauss and Lorentz Size
    """
    h_g_2, h_l_2 = h_g**2, h_l**2
    h_g_3, h_l_3 = h_g_2*h_g, h_l_2*h_l
    h_g_4, h_l_4 = h_g_3*h_g, h_l_3*h_l
    h_g_5, h_l_5 = h_g_4*h_g, h_l_4*h_l
    c_2, c_3, c_4, c_5 = 2.69269, 2.42843, 4.47163, 0.07842
    h_pv = (h_g_5 + c_2*h_g_4*h_l + c_3*h_g_3*h_l_2 + 
            c_4*h_g_2*h_l_3 + c_5*h_g*h_l_4 + h_l_5)**0.2
    hh = h_l*1./h_pv
    eta = 1.36603*hh - 0.47719*hh**2 + 0.11116*hh**3
    return h_pv, eta

In [4]:
tof_obj = rhochi_obj.tof_tof
crystal = rhochi_obj.crystal_cecual
tof_meas = tof_obj.tof_meas
tof_parameters = tof_obj.tof_parameters
tof_profile = tof_obj.tof_profile

In [5]:
cell = crystal.cell

In [6]:
time = tof_meas.numpy_time
d = tof_parameters.calc_d_by_time(time)
d_min, d_max = tof_parameters.calc_d_min_max(time)

In [7]:
sthovl_min = 0.5/d_max
sthovl_max = 0.5/d_min
index_h, index_k, index_l, mult = crystal.calc_hkl(sthovl_min, sthovl_max)
sthovl_hkl = cell.calc_sthovl(index_h, index_k, index_l)
d_hkl = 0.5/sthovl_hkl
time_hkl = tof_parameters.calc_time_by_d(d_hkl)

In [8]:
np_shape_2d = tof_profile.calc_peak_shape_function(
    d, time, time_hkl, size_g=0., strain_g=0.,
    size_l=0., strain_l=0.)

In [9]:
np_shape_2d.max()

nan

False

In [12]:
alpha = tof_profile.alpha0 + tof_profile.alpha1 / d
beta = tof_profile.beta0 + tof_profile.beta1 / d**4
sigma, gamma = calc_sigma_gamma(
    d, tof_profile.sigma0, tof_profile.sigma1, tof_profile.sigma2, tof_profile.gamma0,
    tof_profile.gamma1, tof_profile.gamma2, size_g=0, size_l=0,
    strain_g=0, strain_l=0)

In [20]:
two_over_pi = 2.*numpy.pi
norm = 0.5*alpha*beta/(alpha+beta)
time_2d, time_hkl_2d = numpy.meshgrid(time, time_hkl, indexing="ij")
delta_2d = time_2d-time_hkl_2d

# FIXME: it has to be checked
# sigma = gamma*(inv_8ln2)**0.5
h_pv, eta = calc_hpv_eta(sigma, gamma)
y, z, u, v = calc_y_z_u_v(alpha, beta, sigma, delta_2d) 

exp_u = exp(u)
exp_v = exp(v)
exp_u[numpy.isinf(exp_u)] = 1e200
exp_v[numpy.isinf(exp_v)] = 1e200

profile_g_2d = norm[:, na] * (exp_u * erfc(y) + exp_v * erfc(z))


In [66]:
delta_sec_2d = copy.deepcopy(delta_2d)
delta_sec_2d[delta_2d_sec < -10] = -10
delta_sec_2d[delta_2d_sec > 10] = 10

In [67]:
z1_2d = alpha[:, na]*delta_sec_2d + (1j*0.5*alpha*gamma)[:, na]
z2_2d = -beta[:, na]*delta_sec_2d + (1j*0.5*beta*gamma)[:, na]

In [68]:
imag_fz1_2d = imag(exp1(z1_2d))
imag_fz2_2d = imag(exp1(z2_2d))
# imag_fz1_2d[numpy.isnan(imag_fz1_2d)]=0.
# imag_fz1_2d[numpy.isinf(imag_fz1_2d)]=0.
# imag_fz1_2d[numpy.isneginf(imag_fz1_2d)]=0.
# imag_fz2_2d[numpy.isnan(imag_fz2_2d)]=0.
# imag_fz2_2d[numpy.isinf(imag_fz2_2d)]=0.
# imag_fz2_2d[numpy.isneginf(imag_fz2_2d)]=0.

In [59]:
oml_a_2d = -imag_fz1_2d * two_over_pi
oml_b_2d = -imag_fz2_2d * two_over_pi
profile_l_2d = norm[:, na] * (oml_a_2d + oml_b_2d)

In [69]:
profile_l_2d.min()

-101055.54078529909

In [13]:
norm = 0.5*alpha*beta/(alpha+beta)
time_2d, time_hkl_2d = numpy.meshgrid(time, time_hkl, indexing="ij")
delta_2d = time_2d-time_hkl_2d
y, z, u, v = calc_y_z_u_v(alpha, beta, sigma, delta_2d)

exp_u = exp(u)
exp_v = exp(v)
exp_u[numpy.isinf(exp_u)] = 1e200
exp_v[numpy.isinf(exp_v)] = 1e200


In [14]:
res_2d = norm[:, na] * (exp_u * erfc(y) + exp_v * erfc(z))


In [15]:
(exp_u * erfc(y)).max()

0.04207780664353558

In [16]:
(exp_v * erfc(z)).max()

0.29775157227648913

In [17]:
norm.max()

0.03610976193878971

In [18]:
res_2d.max()

0.010464292574154975